In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import time
from datetime import timedelta
import os
import pandas as pd
import cv2
import urllib
from urllib.error import URLError, HTTPError
import requests

# Functions and classes for loading and using the Inception model.
from inception import inception

In [15]:
inception.maybe_download()

Data has apparently already been downloaded and unpacked.


In [2]:
model = inception.Inception()

## Calculating Transfer Values
Caching the transfer values of the inception model

In [17]:
from inception.inception import transfer_values_cache

In [6]:
file = pd.read_csv('Farfetch/newmaster.csv')
image_links = file.iloc[:,1]
image_links = list(image_links)

#y_true = np.loadtxt('Farfetch/true.csv', delimiter=',', skiprows=0)

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

headers={'User-Agent':user_agent,} 

transfer_values_train = []

bekar = []

for i in range(len(image_links)):
    #print(i)
    pa = str(i+1) + "_mm.jpeg"
    img = cv2.imread('Farfetch master images/' + pa)
    img = np.array(img)
    imgr = cv2.cvtColor( img, cv2.COLOR_RGB2GRAY )
    new_img = np.zeros((len(imgr),len(imgr[0]),3))
    for ii in range(len(imgr)):
        for jj in range(len(imgr[0])):
            new_img[ii][jj][0] = imgr[ii][jj]
            new_img[ii][jj][1] = imgr[ii][jj]
            new_img[ii][jj][2] = imgr[ii][jj]
    img_len = len(new_img.shape)
    if img_len < 3:
        bekar.append(i)
        print("Bekaar")
        continue
    #res = transfer_values_cache(cache_path='inception/cachedparams/inception_clothing_train.pkl',
    #                            images=[img],
    #                            model=model)
    
    res = model.transfer_values(image=new_img)
    
    transfer_values_train.append(res)
    
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [7]:
transfer_values_train = np.array(transfer_values_train)
transfer_values_train.shape

(24548, 2048)

In [9]:
#plt.plot(transfer_values_train[0])
#plt.show()
#plt.plot(transfer_values_train[1])
#plt.show()
np.savetxt("inception/transfer_values_cache_gray.csv", transfer_values_train, delimiter=",")
#file = pd.DataFrame(transfer_values_train)
#file.to_csv('inception/transfer_values_cache_gray_pandas.csv')

In [31]:
classes = ['jackets_blazers', 'underwear_brief_or_boxers', 'trousers_chinos','trousers_cropped', 'jackets_denim',
          'jeans_denim', 'trousers_drop_crotch', 'jackets_leather', 'shorts', 'T-shirt']
n_class = 10
n_dim = 2048

x = tf.placeholder(tf.float32, shape=[None, n_dim], name='x')

y_true_labels = tf.placeholder(tf.float32, shape=[None, n_class], name='y_true_labels')

n_hidden_1 = 2048
n_hidden_2 = 2048

weigths = {
    'h1': tf.Variable(tf.truncated_normal([n_dim, n_hidden_1])),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_class])),
}
biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2': tf.Variable(tf.truncated_normal([n_hidden_2])),
    'out': tf.Variable(tf.truncated_normal([n_class])),
}

layer_1 = tf.add(tf.matmul(x, weigths['h1']), biases['b1'])
layer_1 = tf.nn.relu(layer_1)
    
layer_2 = tf.add(tf.matmul(layer_1, weigths['h2']), biases['b2'])
layer_2 = tf.nn.relu(layer_2)

y_pred_layer = tf.matmul(layer_2, weigths['out']) + biases['out']

y_pred_cls = tf.argmax(y_pred_layer, axis=1, name='y_pred_cls')

y_pred_prbl = tf.nn.softmax(y_pred_layer, name='y_pred_prbl')

cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred_layer,labels=y_true_labels))

learning_rate=1e-4
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost_function)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

global_step = tf.Variable(initial_value=0,
                          name='global_step', trainable=False)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

sess = tf.Session()

sess.run(init)


In [32]:
y_true_chal = np.loadtxt('Farfetch/true.csv', delimiter=',', skiprows=0)

train_batch_size = 64
num_iterations = 10000

def random_batch():
    num_images = len(transfer_values_train)

    idx = np.random.choice(num_images,
                           size=train_batch_size,
                           replace=False)

    # Use the random index to select random x and y-values.
    # We use the transfer-values instead of images as x-values.
    x_batch = transfer_values_train[idx]
    y_batch = y_true_chal[idx]

    return x_batch, y_batch

def optimize():
    start_time = time.time()

    for i in range(num_iterations):
        x_batch, y_true_batch = random_batch()

        feed_dict_train = {x: x_batch,
                           y_true_labels: y_true_batch}

        i_global, _ = sess.run([global_step, optimizer],
                                  feed_dict=feed_dict_train)

        if (i_global % 100 == 0) or (i == num_iterations - 1):
            batch_acc = sess.run(accuracy,
                                    feed_dict=feed_dict_train)

            msg = "Global Step: {0:>6}, Training Batch Accuracy: {1:>6.1%}"
            print(msg.format(i_global, batch_acc))

    end_time = time.time()

    time_dif = end_time - start_time

    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [1]:
#ess.close()
#optimize()